# River telemetry simulation

## Usage:
1. Run the setup cell __once__ (doing so resets parameters to default values)
2. Enter parameters
3. Run simulation cell

In [12]:
# @title Setup cell
from dataclasses import dataclass
import random
from typing import List, Set
import numpy as np
from ipywidgets import interact, widgets
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

random.seed(None)

# Helper function(s)
def create_fish(fish_count: int, river_length: float):
  fish: List[float] = []
  for _ in range(fish_count):
    fish.append(river_length * random.random())

  return fish

# Simulation function
def run_simulation_cases(
    read_radius: float,
    read_time: float,
    fish_count: int,
    river_length: float,
    float_speed: float,
    cases_count: int
):
  fish_cases: List[List[float]] = []
  found_fish_cases: List[Set[int]] = []

  for _ in range(cases_count):
    fish_cases.append(create_fish(fish_count, river_length))
    found_fish_cases.append(set())

  current_frequency: int = 0
  frequency_switch_timer: float = 0

  print('Simulation progress')

  for t in tqdm(np.arange(0.0, river_length, float_speed)):
    if frequency_switch_timer >= read_time:
      if current_frequency == fish_count - 1:
        current_frequency = 0
      else:
        current_frequency += 1

      frequency_switch_timer = 0

    boat_position: float = t

    for i in range(cases_count):
      if abs(fish_cases[i][current_frequency] - boat_position) <= read_radius:
        found_fish_cases[i].add(current_frequency)

    frequency_switch_timer += 1

  results = np.array(list(map(
      lambda s: len(s),
      found_fish_cases
  )))
  return results

# Algebraic estimate
def algebraic_estimate(read_time: float, read_radius: float, float_speed: float):
  return (2 * read_radius) / (float_speed * read_time)

# Setup UI
style = {'description_width': 'initial'}
inputs = [
  widgets.HTML('<h1>Enter simulation parameters</h1>'),
  widgets.FloatText(value=100.0, step=10.0, description="Read radius (m)", style=style),
  widgets.FloatText(value=6.0, description="Frequency read time (s)", style=style),
  widgets.IntText(value=30, description="Number of fish", style=style),
  widgets.FloatText(value=5000.0, step=1000.0, description="Length of river (m)", style=style),
  widgets.FloatText(value=2.0, step=0.5, description="Float speed (m/s)", style=style),
  widgets.IntText(value=100, description="Number of test cases to run", style=style)
]
ui = widgets.VBox(inputs)
display(ui)

In [17]:
# @title Run simulation cell
args = list(map(lambda i: i.value, inputs[1:]))
read_radius = args[0]
read_time = args[1]
fish_count = args[2]
river_length = args[3]
float_speed = args[4]
cases_count = args[5]

data = run_simulation_cases(*args)
print(f'\nData summary (fish found):\nMax: {data.max()}\nMin: {data.min()}\nMean: {data.mean()}\nVariance: {data.var()}\n')

alg_est = algebraic_estimate(read_time, read_radius, float_speed)
print(f'Algebraic estimate: {alg_est}\n')

Simulation progress


  0%|          | 0/2500 [00:00<?, ?it/s]


Data summary (fish found):
Max: 26
Min: 7
Mean: 17.256
Variance: 7.278463999999999

Algebraic estimate: 16.666666666666668

